In [28]:
import plotly.express as px
import pandas as pd

In [20]:
clients = pd.read_pickle("data/parsed/clients.pkl")
transactions= pd.read_pickle("data/parsed/transactions.pkl")

In [21]:
N_SKUIDs = 150 #Filter N most ordered SKUIDs

Client-Order ranked

In [22]:
idx1 = transactions[["POC","ORDER_ID","INVOICE_DATE"]].drop_duplicates()
idx1 = idx1.sort_values(["POC","INVOICE_DATE"])

idx1["ORDER_RANK"] = idx1.groupby("POC")["INVOICE_DATE"].rank(method="first").astype(int)


Client-Order-Product ranked

In [23]:
idx2 = transactions[["POC","ORDER_ID","INVOICE_DATE","SKU_ID"]].drop_duplicates()
idx2 = idx2.sort_values(["POC","INVOICE_DATE","SKU_ID"])

idx2["ORDER_PRODUCT_RANK"] = idx2.groupby(["POC","SKU_ID"])["INVOICE_DATE"].rank(method="first").astype(int)


Build final DF

In [24]:
all_products = transactions[["SKU_ID"]].drop_duplicates()

transactions_ranked = (all_products
    .merge(idx1, how="cross")
    .merge(idx2, on=["POC","ORDER_ID","INVOICE_DATE","SKU_ID"], how="left")
    .merge(transactions, on=["POC","SKU_ID","ORDER_ID","INVOICE_DATE"], how="left")
    .fillna(0)
    [["POC","SKU_ID","ORDER_ID","INVOICE_DATE","ITEMS_PHYS_CASES","ORDER_RANK","ORDER_PRODUCT_RANK"]]
    .sort_values(["POC","SKU_ID","INVOICE_DATE"])
)

transactions_ranked["ORDER_RANK"] = transactions_ranked["ORDER_RANK"].astype(int)
transactions_ranked["ORDER_PRODUCT_RANK"] = transactions_ranked["ORDER_PRODUCT_RANK"].astype(int)

Stage results

In [25]:
transactions_ranked.to_pickle("data/parsed/transactions_ranked.pkl")

Add SKUID rank

In [26]:
target_SKUIDs = (transactions_ranked
    [transactions_ranked["ITEMS_PHYS_CASES"] != 0.0]
    ["SKU_ID"].value_counts()
    .rename("SKUID_counts")
    .reset_index()
    .reset_index()
    .rename(columns={"index":"SKUID_RANK"})
)

transactions_ranked = (transactions_ranked
    .merge(target_SKUIDs, on=["SKU_ID"], how="inner")
    .sort_values(["POC","ORDER_RANK","SKUID_RANK"])
)

Arrange for modeling

In [27]:
transactions_ranked = transactions_ranked[transactions_ranked["SKUID_RANK"] <= N_SKUIDs]

transactions_ranked["target"] = (transactions_ranked["ITEMS_PHYS_CASES"] != 0.0).astype(int)

transactions_ranked = (transactions_ranked
    .set_index(["POC","ORDER_RANK","SKU_ID"])
    ["target"]
    .unstack("SKU_ID")
    .reset_index()
    .sort_values(["POC","ORDER_RANK"])
)


transactions_ranked.to_pickle("data/parsed/modeling_transactions.pkl")